Get data from elasticsearch

In [1]:
from elasticsearch import Elasticsearch
import json
import requests

elasticsearch = Elasticsearch(['http://10.24.240.41:9200/'])
index_list = []

Create query body

In [2]:
sensor = 'sensor-cmc-corp'
query_body = {
  "aggs": {
    "2": {
      "date_histogram": {
        "field": "@timestamp",
        "interval": "5m"
      },
      "aggs": {
        "1": {
          "sum": {
            "field": "IN_BYTES"
          }
        },
        "3": {
          "sum": {
            "field": "OUT_BYTES"
          }
        },
        "4": {
          "sum": {
            "field": "IN_PKTS"
          }
        },
        "5": {
          "sum": {
            "field": "OUT_PKTS"
          }
        }
      }
    }
  },
  "size": 0,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "NTOPNG_INSTANCE_NAME.keyword": "sensor-cmc-corp"
          }
        }
      ]
    }
  }
}
elastic_query = json.dumps(query_body)

Create index list to get data

In [3]:
import datetime

def create_index(date):
    index = 'flows-2019.'
    month = date.month
    if month < 10:
        index += ('0' + str(month))
    else:
        index += str(month)
    day = date.day
    if day < 10:
        index += ('.0' + str(day))
    else:
        index += ('.' + str(day))
    return index

index_list = []
today = datetime.datetime.today
for i in range(9):
    date = datetime.datetime.today() - datetime.timedelta(days=8-i)
    index = create_index(date)
    index_list.append(index)
print(index_list)

['flows-2019.09.30', 'flows-2019.10.01', 'flows-2019.10.02', 'flows-2019.10.03', 'flows-2019.10.04', 'flows-2019.10.05', 'flows-2019.10.06', 'flows-2019.10.07', 'flows-2019.10.08']


Transform datetime to exactly timezone (+7 hours)

In [4]:
import datetime
import maya

def get_true_time_stamp(time_stamp):
    time_stamp = maya.parse(time_stamp).datetime()
    time_stamp = time_stamp + datetime.timedelta(hours=7)
    return str(time_stamp)
get_true_time_stamp('2019-09-04T17:00:00.000Z')

'2019-09-05 00:00:00+00:00'

Get data from index on index list and push to Pandas dataframe

In [5]:
import pandas as pd
data_frame = pd.DataFrame(columns=['TIMESTAMP', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS', 'COUNTS'])

for index in range(1, len(index_list)):
    print(str(index_list[index] + ',' + index_list[index-1]))
    elastic_result = elasticsearch.search(index = str(index_list[index] + ',' + index_list[index-1]), body=elastic_query, request_timeout=60)
    for row in elastic_result['aggregations']['2']['buckets'][17*12 : 17*12+24*12]:
        data_frame = data_frame.append({'TIMESTAMP': get_true_time_stamp(str(row['key_as_string'])), 'IN_BYTES': row['1']['value'], 'OUT_BYTES': row['3']['value'], 'IN_PKTS': row['4']['value'], 'OUT_PKTS': row['5']['value'], 'COUNTS': row['doc_count']}, ignore_index=True)
data_frame.to_csv(sensor + '.csv')

flows-2019.10.01,flows-2019.09.30


ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7fa8b8e1c2b0>: Failed to establish a new connection: [Errno 113] No route to host) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fa8b8e1c2b0>: Failed to establish a new connection: [Errno 113] No route to host)